In [1]:
from Scripts.load_first_data import load_data

df = load_data()

c:\Users\abirm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from Scripts.eda import perform_eda


In [3]:
perform_eda(df)

Lignes: 14640
Colonnes: 13
Distribution sentiments: {'negative': 9178, 'neutral': 3099, 'positive': 2363}


{'total_rows': 14640,
 'total_columns': 13,
 'sentiment_distribution': {'negative': 9178,
  'neutral': 3099,
  'positive': 2363},
 'missing_values': {'tweet_id': 0,
  'airline_sentiment': 0,
  'airline_sentiment_confidence': 0,
  'negativereason': 5462,
  'negativereason_confidence': 4118,
  'airline': 0,
  'name': 0,
  'retweet_count': 0,
  'text': 0,
  'tweet_coord': 13621,
  'tweet_created': 0,
  'tweet_location': 4733,
  'user_timezone': 4820},
 'airline_distribution': {'United': 3822,
  'US Airways': 2913,
  'American': 2759,
  'Southwest': 2420,
  'Delta': 2222,
  'Virgin America': 504}}

In [4]:
from Scripts.normalize import normalize_text

In [5]:
from Scripts.cleaning import clean_text , clean_dataframe

In [6]:
df = normalize_text(df)

In [7]:
df = clean_dataframe(df)

In [8]:
df.head()

,airline_sentiment,airline_sentiment_confidence,airline,text
0,neutral,1.0000,Virgin America,virginamerica what dhepburn said.
1,positive,0.3486,Virgin America,virginamerica plus you've added commercials to...
2,neutral,0.6837,Virgin America,virginamerica i didn't today... must mean i ne...
3,negative,1.0000,Virgin America,virginamerica it's really aggressive to blast ...
4,negative,1.0000,Virgin America,virginamerica and it's a really big bad thing ...


In [9]:
perform_eda(df)

Lignes: 14528
Colonnes: 4
Distribution sentiments: {'negative': 9114, 'neutral': 3087, 'positive': 2327}


{'total_rows': 14528,
 'total_columns': 4,
 'sentiment_distribution': {'negative': 9114,
  'neutral': 3087,
  'positive': 2327},
 'missing_values': {'airline_sentiment': 0,
  'airline_sentiment_confidence': 0,
  'airline': 0,
  'text': 0},
 'airline_distribution': {'United': 3816,
  'US Airways': 2906,
  'American': 2673,
  'Southwest': 2415,
  'Delta': 2215,
  'Virgin America': 503}}

In [32]:
from Scripts.reequilibrage import reequilibrer_donnees
df = reequilibrer_donnees(df)


Taille avant:  27342 lignes
Taille après:  27342 lignes
Ajoutées:      12814 lignes
airline_sentiment
neutral     9114
positive    9114
negative    9114
Name: count, dtype: int64


In [33]:
from Scripts.metadata import prepare_metadata

In [38]:
df['text'] = df['text'].astype(str)


In [39]:
df = prepare_metadata(df)

In [40]:
from Scripts.embedding import generate_embeddings

In [41]:
# 1. Types présents
df['text'].apply(type).value_counts()


text
<class 'str'>    27342
Name: count, dtype: int64

In [42]:
embeddings = generate_embeddings(df['text'].tolist())

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: a34e199c-d36b-47aa-bd19-4403461449fd)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
Batches: 100%|██████████| 428/428 [01:52<00:00,  3.79it/s]


In [43]:
embeddings

array([[-0.10879882,  0.27881262, -0.04908387, ..., -0.29314417,
         0.34666947, -0.00829021],
       [ 0.09001455, -0.28024206,  0.04658645, ..., -0.39974847,
         0.13312009,  0.09550615],
       [ 0.05002017, -0.00446651,  0.27141353, ..., -0.33055556,
        -0.5144448 , -0.51145387],
       ...,
       [ 0.05357482, -0.16622768,  0.06603148, ...,  0.02515328,
         0.04621768,  0.05424234],
       [-0.06787211,  0.10960487,  0.05553555, ..., -0.0022447 ,
         0.08849558,  0.14001021],
       [-0.18171573,  0.05948553, -0.06177798, ..., -0.12337942,
        -0.08097477,  0.1446103 ]], shape=(27342, 384), dtype=float32)

In [44]:
from Scripts.data_split import split_train_test
split_data = split_train_test(df, embeddings)
df_train = split_data['df_train']
df_test = split_data['df_test']
embeddings_train = split_data['embeddings_train']
embeddings_test = split_data['embeddings_test']

In [45]:
from Scripts.chromadb_gen import initialize_chromadb, store_embeddings_train, store_embeddings_test
client = initialize_chromadb(reset=True)
store_embeddings_train(client, df_train, embeddings_train)
store_embeddings_test(client, df_test, embeddings_test)

Collection(name=airline_test)

In [46]:


from Scripts.chromadb_gen import initialize_chromadb, store_embeddings_train, store_embeddings_test
client = initialize_chromadb(reset=True)
store_embeddings_train(client, df_train, embeddings_train)
store_embeddings_test(client, df_test, embeddings_test)





Collection(name=airline_test)

In [47]:
from Scripts.sauvegarde import save_data
save_data(df_train, df_test, embeddings_train, embeddings_test)

In [48]:
import numpy as np
import pandas as pd
X_train = np.load('data/embeddings_train.npy')
X_test = np.load('data/embeddings_test.npy')
df_train = pd.read_csv('data/airline_train.csv')
df_test = pd.read_csv('data/airline_test.csv')
y_train = df_train['label'].values
y_test = df_test['label'].values

In [49]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto', 0.001, 0.01],
}

In [50]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
svm = SVC(random_state=42)
grid_search = GridSearchCV(
    estimator=svm,
    param_grid=param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=2,
)

In [51]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,estimator,SVC(random_state=42)
,param_grid,"{'C': [0.1, 1, ...], 'gamma': ['scale', 'auto', ...], 'kernel': ['linear', 'rbf']}"
,scoring,'f1_weighted'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,10


In [52]:
best_model = grid_search.best_estimator_


In [53]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [54]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

print(f"Meilleurs paramètres: {grid_search.best_params_}")
print(f"\nTrain Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Test F1-Score: {f1_score(y_test, y_test_pred, average='weighted'):.4f}")

Meilleurs paramètres: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

Train Accuracy: 0.9964
Test Accuracy: 0.8859
Test F1-Score: 0.8860


In [55]:
print("\nRapport de classification:")
print(classification_report(y_test, y_test_pred, target_names=['Negative', 'Neutral', 'Positive']))

print("\nMatrice de confusion:")
print(confusion_matrix(y_test, y_test_pred))


Rapport de classification:
              precision    recall  f1-score   support

    Negative       0.87      0.89      0.88      1823
     Neutral       0.87      0.86      0.86      1823
    Positive       0.92      0.90      0.91      1823

    accuracy                           0.89      5469
   macro avg       0.89      0.89      0.89      5469
weighted avg       0.89      0.89      0.89      5469


Matrice de confusion:
[[1630  151   42]
 [ 154 1569  100]
 [  84   93 1646]]


In [56]:
import joblib
import os
os.makedirs('models', exist_ok=True)
joblib.dump(best_model, 'models/svm_best_model.pkl')

['models/svm_best_model.pkl']